# Reconnaissance d'entités nommées avec SpaCy

La documentation est accessible ici: https://spacy.io/api

## Imports

In [2]:
from collections import defaultdict
import spacy
from spacy.lang.fr.examples import sentences
!python -m spacy download fr_core_news_md

     ---------------------------------------- 0.0/45.8 MB ? eta -:--:--
     - -------------------------------------- 1.3/45.8 MB 6.7 MB/s eta 0:00:07
     -- ------------------------------------- 2.9/45.8 MB 6.7 MB/s eta 0:00:07
     --- ------------------------------------ 4.2/45.8 MB 6.6 MB/s eta 0:00:07
     ---- ----------------------------------- 5.2/45.8 MB 6.2 MB/s eta 0:00:07
     ----- ---------------------------------- 6.6/45.8 MB 6.0 MB/s eta 0:00:07
     ------- -------------------------------- 8.4/45.8 MB 6.5 MB/s eta 0:00:06
     -------- ------------------------------- 10.2/45.8 MB 6.6 MB/s eta 0:00:06
     ---------- ----------------------------- 11.8/45.8 MB 6.7 MB/s eta 0:00:06
     ----------- ---------------------------- 13.4/45.8 MB 6.7 MB/s eta 0:00:05
     ------------ --------------------------- 14.7/45.8 MB 6.6 MB/s eta 0:00:05
     -------------- ------------------------- 16.3/45.8 MB 6.7 MB/s eta 0:00:05
     -------------- ------------------------- 17.0/45.


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
nlp = spacy.load('fr_core_news_md')

## Exemple sur un corpus de test fourni par SpaCy

In [4]:
# Imprimer le corpus de Spacy
sentences

['Apple cherche à acheter une start-up anglaise pour 1 milliard de dollars',
 "Les voitures autonomes déplacent la responsabilité de l'assurance vers les constructeurs",
 "San Francisco envisage d'interdire les robots coursiers sur les trottoirs",
 'Londres est une grande ville du Royaume-Uni',
 'L’Italie choisit ArcelorMittal pour reprendre la plus grande aciérie d’Europe',
 "Apple lance HomePod parce qu'il se sent menacé par l'Echo d'Amazon",
 "La France ne devrait pas manquer d'électricité cet été, même en cas de canicule",
 'Nouvelles attaques de Trump contre le maire de Londres',
 'Où es-tu ?',
 'Qui est le président de la France ?',
 'Où est la capitale des États-Unis ?',
 'Quand est né Barack Obama ?']

In [5]:
# Isoler la première phrase
sent = sentences[0]
sent

'Apple cherche à acheter une start-up anglaise pour 1 milliard de dollars'

In [6]:
# Traiter la phrase avec Spacy
doc = nlp(sent)

In [7]:
type(doc)

spacy.tokens.doc.Doc

In [8]:
doc.text

'Apple cherche à acheter une start-up anglaise pour 1 milliard de dollars'

In [9]:
doc.to_json()

{'text': 'Apple cherche à acheter une start-up anglaise pour 1 milliard de dollars',
 'ents': [{'start': 0, 'end': 5, 'label': 'ORG'}],
 'sents': [{'start': 0, 'end': 72}],
 'tokens': [{'id': 0,
   'start': 0,
   'end': 5,
   'tag': 'PROPN',
   'pos': 'PROPN',
   'morph': 'Gender=Masc|Number=Sing',
   'lemma': 'Apple',
   'dep': 'nsubj',
   'head': 1},
  {'id': 1,
   'start': 6,
   'end': 13,
   'tag': 'VERB',
   'pos': 'VERB',
   'morph': 'Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin',
   'lemma': 'cherche',
   'dep': 'ROOT',
   'head': 1},
  {'id': 2,
   'start': 14,
   'end': 15,
   'tag': 'ADP',
   'pos': 'ADP',
   'morph': '',
   'lemma': 'à',
   'dep': 'mark',
   'head': 3},
  {'id': 3,
   'start': 16,
   'end': 23,
   'tag': 'VERB',
   'pos': 'VERB',
   'morph': 'VerbForm=Inf',
   'lemma': 'acheter',
   'dep': 'xcomp',
   'head': 1},
  {'id': 4,
   'start': 24,
   'end': 27,
   'tag': 'DET',
   'pos': 'DET',
   'morph': 'Definite=Ind|Gender=Fem|Number=Sing|PronType=Art'

In [10]:
# Appliquer le test sur toutes les phrases
for sent in sentences:
    doc = nlp(sent)
    entities = []
    for ent in doc.ents:
        entities.append(f"{ent.text} ({ent.label_})")
    if entities:
        print(f"'{doc.text}' contient les entités suivantes : {', '.join(entities)}")
    else:
        print(f"'{doc.text}' ne contient aucune entité")

'Apple cherche à acheter une start-up anglaise pour 1 milliard de dollars' contient les entités suivantes : Apple (ORG)
'Les voitures autonomes déplacent la responsabilité de l'assurance vers les constructeurs' ne contient aucune entité
'San Francisco envisage d'interdire les robots coursiers sur les trottoirs' contient les entités suivantes : San Francisco (LOC)
'Londres est une grande ville du Royaume-Uni' contient les entités suivantes : Londres (LOC), Royaume-Uni (LOC)
'L’Italie choisit ArcelorMittal pour reprendre la plus grande aciérie d’Europe' contient les entités suivantes : L’Italie (LOC), ArcelorMittal (ORG), Europe (LOC)
'Apple lance HomePod parce qu'il se sent menacé par l'Echo d'Amazon' contient les entités suivantes : Apple (ORG), HomePod (MISC), Echo (ORG), Amazon (ORG)
'La France ne devrait pas manquer d'électricité cet été, même en cas de canicule' contient les entités suivantes : La France (LOC)
'Nouvelles attaques de Trump contre le maire de Londres' contient les en

## Appliquer la reconnaissance d'entités nommées sur notre corpus

In [11]:
# Charger le texte
n=1000000
text = open("../data/all.txt", encoding='utf-8').read()[:n]

In [12]:
%%time
# Traiter le texte

doc = nlp(text)

CPU times: total: 2min 28s
Wall time: 7min 44s


In [13]:
# Compter les entités
people = defaultdict(int)
for ent in doc.ents:
    if ent.label_ == "PER" and len(ent.text) > 3:
        people[ent.text] += 1

In [14]:
# Trier et imprimer

sorted_people = sorted(people.items(), key=lambda kv: kv[1], reverse=True)

for person, freq in sorted_people[:50]:
    print(f"{person} apparait {freq} fois dans le corpus")

Messieurs apparait 30 fois dans le corpus
Verger apparait 12 fois dans le corpus
Pape apparait 10 fois dans le corpus
Créd apparait 9 fois dans le corpus
Verhaegen apparait 7 fois dans le corpus
Belg apparait 7 fois dans le corpus
M. Malou apparait 6 fois dans le corpus
Holl apparait 6 fois dans le corpus
Van Zeeland apparait 6 fois dans le corpus
Robert apparait 5 fois dans le corpus
Lebrun apparait 5 fois dans le corpus
M. Genay apparait 5 fois dans le corpus
Lundi apparait 4 fois dans le corpus
Tisza apparait 4 fois dans le corpus
Mme Loubet apparait 4 fois dans le corpus
Monsieur apparait 4 fois dans le corpus
Président apparait 4 fois dans le corpus
Mme de Sermaize apparait 4 fois dans le corpus
M. Van apparait 4 fois dans le corpus
Arcy apparait 4 fois dans le corpus
Adieu apparait 4 fois dans le corpus
Thaddée apparait 4 fois dans le corpus
Armand apparait 4 fois dans le corpus
Mgr Lecouvet apparait 4 fois dans le corpus
Staline apparait 4 fois dans le corpus
Lord Lloyd apparait

Exercice: essayez de lister les lieux (LOC) et les organisations (ORG) les plus mentionnées dans le corpus